In [26]:
#Trying to improve the efficiency over ErrorStudy2

In [27]:
#import packages etc
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy
import re
from elasticsearch import Elasticsearch
matplotlib.rc('font', **{'size': 15})
from elasticsearch.helpers import scan
import time
import collections

In [28]:
#declare variables
print("Declaring variables.")

#epoch times of start and end of time interval. 
#first week of february. monday 30. Jan to sunday 5. feb.
#found using epochconverter.com
#epoch time must be in milliseconds
#epoch_start = 1488668400000 #Mar 5th 00.00.00 (week 10)
epoch_start =1485730800000 #jan 30th 00.00.00
#epoch_end = 1489273199000 #mar 11th 23.59.59
epoch_end = 1486335599000 #feb 5th 23.59.59
#epoch_end = 1485820799000 # jan30th 23.59.59 (one day for testing)
#epoch_end = 1485920799000

loop_time = epoch_start #time variable in loop

#loop time increments . one minute? five minutes?
increment = (5 * 60 * 1000)
timevector = [epoch_start]
dummy = epoch_start
while dummy <= epoch_end:
    dummy += increment
    timevector.append(dummy)


#thresholds for defining an error
failure_rate = .5 #failure rate threshold. Q: what is the threshold of red cells in DDM dashboard?
file_thresh = 10 #number of files threshold.
persist_thresh = 15 * 60 * 1000; #milliseconds. time needed before alert is sent.

#number of alarms
alarms = 0

#date and time format of
#ex: 2017-03-27 10:03:10
pattern = '%Y-%m-%d %H:%M:%S'



#some kind of dictionary with pairs of sites, and some variable keeping track of time
#if pairs stay on this list too long an alarm is raised {[source,destination]:interval}
observed_endpoints = {}

#dict of when an alert was made and what pair is failing {when: [source,receiver]}
print("Done.")

Declaring variables.
Done.


In [29]:
#Elasticsearch querries
print("Defining ElasticSearch querries.")
query_done = {
    "size": 0,
    "query": {
    "bool": {
    "must": [
    {
        "range": {
        "@timestamp": {
            "gte": epoch_start,
            "lte": epoch_end,
            "format": "epoch_millis"
            }
        }
    },
        {"term": {"event_type": "transfer-done"}},
        {"term": {"payload.scope": "data16_13TeV"}}
    ]
    }
    }
}
query_failed = {
    "size": 0,
    "query": {
    "bool": {
    "must": [
    {
        "range": {
            "@timestamp": {
            "gte": epoch_start,
            "lte": epoch_end,
            "format": "epoch_millis"
             }
            }
           },
          {"term": {"event_type": "transfer-failed"}},
          {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
    }
}


#make scrolls
print("Defining scrolls.")
# make scrolls
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60) 
my_index = "rucio-events-2017*"
scroll_done = scan(es, query=query_done, index=my_index, scroll='5m', timeout="5m", size=100)
scroll_errors = scan(es, query=query_failed, index=my_index, scroll='5m', timeout="5m", size=100)


print("Done.")

Defining ElasticSearch querries.
Defining scrolls.
Done.


In [30]:
#could be useful to track how much time is spent on this block
time_tracker = time.time()

interval_transfers = {}
interval_failures = {}
interval_ratios = {}
#for interval in timevector:
#    interval_transfers[interval] = {}
#    interval_failures[interval] = {}
#    interval_ratios[interval] = {}

    
transfer_times = {}
print('Starting Loop over Scroll Transfers')
for entry in scroll_done:
    submitted_at = entry['_source']['payload']['submitted_at'] 
    #convert submitted at to epoch time
    #must be in milliseconds
    submitted_at = int(time.mktime(time.strptime(submitted_at, pattern)))*1000
    source = entry['_source']['payload']['src-rse']
    destination = entry['_source']['payload']['dst-rse']
    pair = (source,destination)
    transfer_times[submitted_at] = pair
print('Loop over Scroll Transfers Done')


#sort transfers by time
sorted_transfer_times = collections.OrderedDict(sorted(transfer_times.items()))
#sorted(endpointPairs.items(), key=operator.itemgetter(1))


print('Constructing Transfer Dict')
bookkeeper = list(sorted_transfer_times.items())[0][0] #first key of sorted_trasfer_times
interval_transfers[bookkeeper] = {}
interval_failures[bookkeeper] = {} #ensure that transfers and failures are same length
for entry in sorted_transfer_times.keys():
    if entry - bookkeeper <= increment:
        if sorted_transfer_times[entry] in interval_transfers[bookkeeper].keys():
             interval_transfers[bookkeeper][sorted_transfer_times[entry]] = interval_transfers[bookkeeper][sorted_transfer_times[entry]] + 1
        if sorted_transfer_times[entry] not in interval_transfers[bookkeeper].keys():
             interval_transfers[bookkeeper][sorted_transfer_times[entry]] = 1
    else:
            bookkeeper = bookkeeper+increment
            interval_transfers[bookkeeper] = {}
            interval_failures[bookkeeper] = {}
            
print('Transfer Dict Constructed')
    
#print(interval_transfers)

failure_times  = {}
print('Starting Loop over Scroll Errors')
for entry in scroll_errors:
    submitted_at = entry['_source']['payload']['submitted_at'] 
    #convert submitted at to epoch time
    #must be in milliseconds
    submitted_at = int(time.mktime(time.strptime(submitted_at, pattern)))*1000
    source = entry['_source']['payload']['src-rse']
    destination = entry['_source']['payload']['dst-rse']
    pair = (source,destination)
    failure_times[submitted_at] = pair
print('Loop over Scroll Errors Done')

sorted_failure_times = collections.OrderedDict(sorted(failure_times.items()))

#trim start of sorted failure_times such that times are before first entry of sorted_transfer_times.
#also remove entries in failures if they fall after last entry of transfers
for entry in sorted_failure_times.keys():
    if list(sorted_transfer_times.items())[0][0] - entry > 0 or entry - list(sorted_transfer_times.items())[0][-1] >0: 
        del entry
    else:
        break

print('first transfer: ', list(sorted_transfer_times.items())[0][0])
print('first failure: ', list(sorted_failure_times.items())[0][0])

print('last transfer: ', list(sorted_transfer_times.items())[0][-1])
print('last failure: ', list(sorted_failure_times.items())[0][-1])


print('Constructing Failures Dict')
bookkeeper = list(sorted_transfer_times.items())[0][0] #first key of sorted_trasfer_times
for entry in sorted_failure_times.keys():
    if entry - bookkeeper <= increment:
        if sorted_failure_times[entry] in interval_failures[bookkeeper].keys():
            interval_failures[bookkeeper][sorted_failure_times[entry]] = interval_failures[bookkeeper][sorted_failure_times[entry]] + 1
        if sorted_failure_times[entry] not in interval_failures[bookkeeper].keys():
            interval_failures[bookkeeper][sorted_failure_times[entry]] = 1
    else:
        bookkeeper = bookkeeper+increment
        interval_failures[bookkeeper] = {}
        
print('Failures Dict Constructed')

print('Calculating Failure Ratios')
#calculating the file transfer failure rate betwen pairs of grid sites in each time interval
for interval in interval_transfers:
    interval_ratios[interval] = {}
    for pair in interval_transfers[interval].keys():
        if pair in interval_failures[interval].keys():
            interval_ratios[interval][pair] = float(interval_failures[interval][pair])/(float(interval_failures[interval][pair])+float(interval_transfers[interval][pair]))
        else:
            interval_ratios[interval][pair] = 0.0
    for pair in interval_failures[interval].keys():
        if pair not in interval_transfers[interval].keys():
            interval_ratios[interval][pair] = 1.0
    
    
print('Failure Ratios Calulated')

print('Starting Alarm Loop')

alarm_list = []

#observed_endpoints
for interval in interval_ratios:
    remove = []
    for pair in interval_ratios[interval].keys():
        #number of total failed and successful transfer betwen pair
        transfers = 0
        if pair in interval_transfers[interval].keys():
            transfers += interval_transfers[interval][pair]
        if pair in interval_failures[interval].keys():
            transfers += interval_failures[interval][pair]
        #failure rate must be over threshold and numer of transfers over threshold
        if(interval_ratios[interval][pair] >= failure_rate and transfers >= file_thresh):
            if pair not in observed_endpoints.keys():
                observed_endpoints[pair] = interval
            if pair in observed_endpoints.keys() and interval - observed_endpoints[pair]  >= persist_thresh:
                #print('alarm')
                alarms += 1
                alarm_list.append([interval,pair])
                del observed_endpoints[pair]
                
    # if not in ratios or threshold too low, remove from watchlist
        if(pair in observed_endpoints.keys() and (interval_ratios[interval][pair] <= failure_rate or transfers < file_thresh)):
            
            del observed_endpoints[pair]
            
                        
print('Alarm Loop Done')

print("time spent: ", (time.time() - time_tracker)/60, "minutes.")  
print('Alarms: ', alarms)

Starting Loop over Scroll Transfers
Loop over Scroll Transfers Done
Constructing Transfer Dict
Transfer Dict Constructed
Starting Loop over Scroll Errors
Loop over Scroll Errors Done


TypeError: unsupported operand type(s) for -: 'int' and 'tuple'

In [ ]:
for entry in alarm_list:
    print('When: ',time.strftime(pattern,time.localtime(entry[0]/1000)),', pair: ',entry[1])